# Plate solving fits file

## import modules

우선 필요한 모듈을 import합니다.

In [7]:
from pathlib import Path
from astropy.io import fits

import ysfitsutilpy as yfu

import _astro_utilities
import _Python_utilities

import warnings
warnings.filterwarnings('ignore')

## 환경 설정

In [8]:
#%%
verbose =  False
tryagain = False
trynightsky = True
#######################################################
BASEDIR = Path("/mnt/Rdata/ASTRO_data")  

PROJECDIR = BASEDIR / "C1-Variable"
TODODIR = PROJECDIR / "-_-_-_2017-01_-_RiLA600_STX-16803_-_2bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
if verbose == True :
    print ("DOINGDIRs: ", format(DOINGDIRs))
    print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

try : 
    BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
    if verbose == True :
        print ("BDFDIR: ", format(BDFDIR))
    BDFDIR = Path(BDFDIR[0])    
except : 
    BDFDIR = TODODIR
    pass

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
if verbose == True :
    print ("DOINGDIRs: ", DOINGDIRs)
    print ("len(DOINGDIRs): ", len(DOINGDIRs))
#######################################################

DOINGDIRs:  ['/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/-_CAL-BDF_-_2024-05_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-03_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-31_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-37b_LIGHT_-_2024-06-28_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HD189733b_LIGHT_-_2024-06-16_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HD189733b_LIGHT_-_2024-06-27_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/Kepler-17b_LIGHT_-_2024-06-26_-_GSON300_STF-8300M_-_1bin/', '/mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/Qatar-10b_LIGH

## plate solving

``DOINGDIRs``에 list 형태로 들어있는 디렉토리 경로안에 모든 파일들을 ``yfu.make_summary`` 함수를 이용하여 dataframe으로 만들어 파일명 하나하나 ``platesolving``을 수행합니다. 

In [ ]:
for DOINGDIR in DOINGDIRs[:1] :
    DOINGDIR = Path(DOINGDIR)
    if verbose == True :
        print("DOINGDIR", DOINGDIR)
    SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_dir
    SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_nightsky_dir
    SOLVINGDIR = DOINGDIR


    summary = yfu.make_summary(SOLVINGDIR/"*.fit*")
    if summary is not None :
        if verbose == True :
            print("len(summary):", len(summary))
            print("summary:", summary)
            #print(summary["file"][0])  
        df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
        df_light = df_light.reset_index(drop=True)
        if verbose == True :
            print("df_light:\n{}".format(df_light))
    df_light

    for _, row  in df_light.iterrows():

        fpath = Path(row["file"])
        if verbose == True :
            print("fpath :" ,fpath)
        hdul = fits.open(fpath)

        if 'PIXSCALE' in hdul[0].header:
            PIXc = hdul[0].header['PIXSCALE']
        else : 
            PIXc = _astro_utilities.calPixScale(hdul[0].header['FOCALLEN'], 
                                                hdul[0].header['XPIXSZ'],
                                                hdul[0].header['XBINNING'])
        if verbose == True :
            print("PIXc : ", PIXc)
        hdul.close()

        solved = _astro_utilities.KevinSolver(fpath, 
                                                #str(SOLVEDDIR), 
                                                # downsample = 2,
                                                # pixscale = PIXc,
                                                tryASTAP = True, 
                                                tryLOCAL = False,
                                                makeLOCALsh = True,
                                                tryASTROMETRYNET = False, 
                                                verbose = verbose,
                                                )

All 71 keywords (guessed from /mnt/Rdata/ASTRO_data/C3-EXO/-_-_-_2024-06_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_-_2024-08-03_-_GSON300_STF-8300M_-_1bin/HAT-P-23b_LIGHT_R_2024-08-03-12-01-12_120sec_GSON300_STF-8300M_29c_1bin.fit) will be loaded.


len(summary): 122
